In [6]:
import re

In [3]:
import pandas as pd
from itertools import combinations
import re
import os

In [19]:
names = pd.read_csv('lotr_names_clean.csv')

In [20]:
names_lst = list(set(names['names']))

In [21]:
len(names_lst)

98

In [8]:
names_lst[:2]

['Merry', 'Théoden']

In [22]:
pairwise_characters = list(combinations(names_lst, 2))
pairwise_characters_list = [list(pair) for pair in pairwise_characters]


In [10]:
books = ["hobbit.txt",
        "The Fellowship Of The Ring.txt",
          "The Two Towers.txt",
          "The Return Of The King.txt"
         ]

In [23]:
for book in books:
    print(book)
    dir_name = book.split('.')[0] + '_2'
    os.makedirs(dir_name, exist_ok=True)

    pairwise_characters = pairwise_characters_list

    with open(book, 'r') as file:
        content = file.read()
        content = content.replace("\n", " ")
        content = content.replace('\u00A0', ' ').strip()
        content = content.replace('"', ' ')
        content = content.replace("Meriadoc", "Merry").replace("Peregrin", "Pippin").replace("Samwise", "Sam")
        content = content.replace("Smeagol", "Gollum")
        sentences = re.split(r'(?<=[.!?])\s+', content)

    for pair in pairwise_characters:
        story = []
        weight = 0
        idx = 0

        if pair[0] == 'Pippin' or pair[1] == 'Pippin':
            while idx < len(sentences):
                if pair[0] in sentences[idx]:
                    if pair[1] in " ".join(sentences[idx-2:idx+3]):
                        weight += 1
                        story.append(" ".join(sentences[idx-2:idx+3]))
                        idx +=4
                    else:
                        idx +=1
                else:
                    idx +=1



            if weight >= 3:
                file_name = pair[0] + '_' + pair[1] + '_' + '.csv'
                file_path = os.path.join(dir_name, file_name)
                story_to_save = pd.DataFrame(story, columns = ['phrases'])
                story_to_save.to_csv(file_path, index = False)


hobbit.txt
The Fellowship Of The Ring.txt
The Two Towers.txt
The Return Of The King.txt


In [12]:
!pip install -q transformers

In [13]:
from transformers import pipeline

In [14]:
sentiment_pipeline = pipeline("sentiment-analysis")
emotion_pipeline = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [15]:
def get_labels(df):

    sentiment = sentiment_pipeline(list(df['phrases']))
    phrase_sentiment = pd.concat([df, pd.DataFrame(sentiment)['label']], axis=1)
    #print(phrase_sentiment.head())

    phrase_emotion = emotion_pipeline(list(df['phrases']) )
    df_emotion = pd.DataFrame([max(i, key=lambda x: x['score'])["label"] for i in phrase_emotion], columns = ['emotion'])

    df_full = pd.concat([phrase_sentiment, df_emotion] , axis=1)

    return df_full

In [24]:
folders = ["hobbit_2",
        "The Fellowship Of The Ring_2", "The Two Towers_2", "The Return Of The King_2"]

df_final = pd.DataFrame()

classify = True
final_output = "lotr_network_classified_pippin.csv"

for folder_path in folders:
    for filename in os.listdir(folder_path):
        print(filename)
        #if os.path.isfile(os.path.join(folder_path, filename)):

        pair = pd.read_csv(os.path.join(folder_path, filename))

        if classify:
            story = get_labels(pair)
            #story.to_csv(os.path.join(folder_path, filename), index = False)
        else:
            story = pair.copy()

        meta_df = pd.DataFrame({"book" : [folder_path],
                        "person_1" : [filename.split('_')[0]],
                        "person_2" : [filename.split('_')[1]],
                        })

        story['key'] = 0
        meta_df['key'] = 0
        pair_df_meta = pd.merge(story, meta_df, on='key').drop('key', axis=1)

        df_final = pd.concat([df_final, pair_df_meta], axis=0, ignore_index=True)
    #     break
    # break


df_final.to_csv(final_output, index = False)
df_final.shape

Pippin_Farmer_.csv
Pippin_Bilbo_.csv
Merry_Pippin_.csv
Gaffer_Pippin_.csv
Pippin_Fredegar_.csv
Pippin_Legolas_.csv
Pippin_Gimli_.csv
Pippin_Sam_.csv
Pippin_Aragorn_.csv
Pippin_Boromir_.csv
Pippin_Gandalf_.csv
Pippin_Elrond_.csv
Frodo_Pippin_.csv
Tom_Pippin_.csv
Pippin_Folco_.csv
Merry_Pippin_.csv
Théoden_Pippin_.csv
Pippin_Legolas_.csv
Pippin_Gimli_.csv
Pippin_Sam_.csv
Pippin_Saruman_.csv
Pippin_Aragorn_.csv
Pippin_Boromir_.csv
Pippin_Gandalf_.csv
Frodo_Pippin_.csv
Pippin_Treebeard_.csv
Pippin_Paladin_.csv
Pippin_Faramir_.csv
Pippin_Bilbo_.csv
Merry_Pippin_.csv
Pippin_Éomer_.csv
Théoden_Pippin_.csv
Pippin_Legolas_.csv
Pippin_Gimli_.csv
Pippin_Galadriel_.csv
Pippin_Sam_.csv
Pippin_Lotho_.csv
Pippin_Denethor_.csv
Pippin_Saruman_.csv
Pippin_Aragorn_.csv
Pippin_Boromir_.csv
Pippin_Gandalf_.csv
Frodo_Pippin_.csv


(905, 6)